# Random Forest Regression from Scratch

Group 18 Members:

- Clara Pichler, 11917694
- Hannah Knapp, 11901857 
- Sibel Toprakkiran, 09426341

### Overview

1. Bootstraping
- `make_bootstraps(df, n_bootstraps=100)`

2. Decision Tree Regression
- `mse` or `variance reduction`
- `split_dataset(X, y, feature_idx, threshold)`
- `find_best_split(X, y)`
- `build_tree(X, y, max_depth, min_samples_split, depth=0)`
- `predict_tree(tree, X)`

3. Random Forest Regression

4. Random Forest Regression - LLM
- 

5. Evaluation
- Ours
- LLM
- sklearn

In [297]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

We will use the data set `mountains_vs_beaches_preferences.csv` as a data frame for testing our functions.

In [298]:
df_pref = pd.read_csv('data/mountains_vs_beaches_preferences.csv')

df_pref_one_hot = pd.get_dummies(df_pref, columns=['Location', 'Favorite_Season', 'Gender'], drop_first=True)

target_mean = df_pref_one_hot.groupby('Preferred_Activities')['Preference'].mean()
df_pref_one_hot['Preferred_Activities_Encoded'] = df_pref_one_hot['Preferred_Activities'].map(target_mean)

education_mapping = {'high school': 0, 'bachelor': 1, 'master': 2, 'doctorate': 3}
df_pref_one_hot['Education_Level_Encoded'] = df_pref_one_hot['Education_Level'].map(education_mapping)

df_pref_one_hot = df_pref_one_hot.drop(["Education_Level", "Preferred_Activities"], axis=1)

display(df_pref_one_hot.info())
display(df_pref_one_hot.isnull().sum())
display(df_pref_one_hot.head())

display(df_pref_one_hot['Preference'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52444 entries, 0 to 52443
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           52444 non-null  int64  
 1   Income                        52444 non-null  int64  
 2   Travel_Frequency              52444 non-null  int64  
 3   Vacation_Budget               52444 non-null  int64  
 4   Proximity_to_Mountains        52444 non-null  int64  
 5   Proximity_to_Beaches          52444 non-null  int64  
 6   Pets                          52444 non-null  int64  
 7   Environmental_Concerns        52444 non-null  int64  
 8   Preference                    52444 non-null  int64  
 9   Location_suburban             52444 non-null  bool   
 10  Location_urban                52444 non-null  bool   
 11  Favorite_Season_spring        52444 non-null  bool   
 12  Favorite_Season_summer        52444 non-null  bool   
 13  F

None

Age                             0
Income                          0
Travel_Frequency                0
Vacation_Budget                 0
Proximity_to_Mountains          0
Proximity_to_Beaches            0
Pets                            0
Environmental_Concerns          0
Preference                      0
Location_suburban               0
Location_urban                  0
Favorite_Season_spring          0
Favorite_Season_summer          0
Favorite_Season_winter          0
Gender_male                     0
Gender_non-binary               0
Preferred_Activities_Encoded    0
Education_Level_Encoded         0
dtype: int64

,Age,Income,Travel_Frequency,Vacation_Budget,Proximity_to_Mountains,Proximity_to_Beaches,Pets,Environmental_Concerns,Preference,Location_suburban,Location_urban,Favorite_Season_spring,Favorite_Season_summer,Favorite_Season_winter,Gender_male,Gender_non-binary,Preferred_Activities_Encoded,Education_Level_Encoded
0,56,71477,9,2477,175,267,0,1,1,False,True,False,True,False,True,False,0.500114,1
1,69,88740,1,4777,228,190,0,1,0,True,False,False,False,False,True,False,0.000000,2
2,46,46562,0,1469,71,280,0,0,1,False,True,False,False,True,False,False,0.500114,2
3,32,99044,6,1482,31,255,1,0,1,False,False,False,True,False,False,True,0.500916,0
4,60,106583,5,516,23,151,1,1,0,True,False,False,False,True,False,False,0.000000,0


Preference
0    39296
1    13148
Name: count, dtype: int64

## Abalone dataset

In [299]:
url='./data/abalone.csv'
column_names = ["Sex", "Length", "Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight", "Rings"]
abalone_df = pd.read_csv(url, header=0, names=column_names)
abalone_df[abalone_df.Height == 0]
abalone_df = abalone_df[abalone_df.Height != 0]
df_abalone = pd.get_dummies(abalone_df, columns=['Sex'], drop_first=False)
df_abalone.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,False,False,True
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,False,False,True
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,True,False,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,False,False,True
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,False,True,False


In [300]:
df_airfol = pd.read_csv("./data/airfoil_noise_data.csv")

display(df_airfol.info())
display(df_airfol.isnull().sum())
display(df_airfol.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0      1503 non-null   int64  
 1   x1      1503 non-null   float64
 2   x2      1503 non-null   float64
 3   x3      1503 non-null   float64
 4   x4      1503 non-null   float64
 5   y       1503 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 70.6 KB


None

x0    0
x1    0
x2    0
x3    0
x4    0
y     0
dtype: int64

,x0,x1,x2,x3,x4,y
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


## Bootstrapping

Bootstrapping is a method to create multiple subsets of the original data by sampling with replacement. Each subset is used to train one decision tree in the forest.

- Introduces randomness, ensuring that trees see different views of the data.
- Helps in reducing overfitting by decorrelating the trees.

- __Sample Size__: The size of the bootstrap sample is the same as the original dataset.
- __Replacement__: Sampling with replacement ensures diversity between bootstrapped samples.


In [301]:
def make_bootstraps(X, y, n_bootstraps=100):
    
    bootstrap = []
    sample_size = X.shape[0]
    idx = np.arange(sample_size)

    for b in range(n_bootstraps):
        
        sidx = np.random.choice(idx, size=sample_size, replace=True)
        X_boot = X[sidx]
        y_boot = y[sidx]
        
        bootstrap.append((X_boot, y_boot))
        
    return bootstrap

How our function `make_bootstraps` works:
1. Each data point has equal probability of being selected 
2. Selecting data points from the original sample for the current bootstrap sample, with replacement! Until we reached the same size as the original data
4. Repeating this process until we have `n_bootstraps` bootstrap samples which we save in a dictonary `dic_boot`

In [302]:
X = df_pref_one_hot.drop("Preference", axis=1).values
y = df_pref_one_hot["Preference"].values

dic_boot_pref = make_bootstraps(X, y)

## Decision Tree Regression

First, we define the functions to split the data and calculate metrics like the mean squared error (MSE).
And for the actual building of the trees we recursively split the data into smaller groups, based on feature thresholds, until a stopping condition is met (e.g., max depth or minimum samples per leaf).


In [303]:
def mse(y):
    return np.mean((y - np.mean(y))**2)

def variance_reduction(parent, leftChild, rightChild):    
        weight_l = len(leftChild) / len(parent)
        weight_r = len(rightChild) / len(parent)
        var_red = np.var(parent) - (weight_l * np.var(leftChild) + weight_r * np.var(rightChild))
        return var_red

`mse(y)` calculates how spread out the data points are from their mean. A lower MSE indicates that the values in `y` are closer to the mean, which means the split effectively reduces variability. 

`variance reduction(parent, leftChild, rightChild)` calculates the variance of the parent node and subtracts the weighted variance of the child nodes. Later we want to choose the split with the highest variance reduction.

In [304]:
def split_dataset(X, y, feature_idx, threshold):
    
    left_mask = X[:, feature_idx] <= threshold
    right_mask = ~left_mask

    return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

`split_dataset` divides X (features) and y (target) into two groups based on whether the value of a feature is less than or equal to a given threshold. This is used to evaluate potential splits during training. For example split on feature `Unemployment` at value 6 creates two groups: rows where `Unemployment <= 6` and rows where `Unemployment > 6`.

In [305]:
def find_best_split(X, y, split_measure, feature_subset=None):

    best_feature, best_threshold = None, None
    best_mse = float("inf")
    best_var_red = float("-inf")
    features = feature_subset if feature_subset is not None else range(X.shape[1])
    
    for feature_idx in features:
        thresholds = np.unique(X[:, feature_idx])
        
        for threshold in thresholds:
            _, _, y_left, y_right = split_dataset(X, y, feature_idx, threshold)
            
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            
            if split_measure == 'variance':
                var_red = variance_reduction(y, y_left, y_right)
                if var_red > best_var_red:
                    best_var_red = var_red
                    best_feature = feature_idx
                    best_threshold = threshold
                    
            elif split_measure == 'mse':
                mse_split = (len(y_left) * mse(y_left) + len(y_right) * mse(y_right)) / len(y)
            
                if mse_split < best_mse:
                    best_mse = mse_split
                    best_feature = feature_idx
                    best_threshold = threshold
                
    return best_feature, best_threshold

`find_best_split` evaluates all possible splits for every feature and every threshold. It selects the split that minimizes the weighted MSE of the left and right groups:
$$
  \text{Weighted MSE} = \frac{n_{\text{left}}}{n_{\text{total}}} \cdot \text{MSE(left)} + \frac{n_{\text{right}}}{n_{\text{total}}} \cdot \text{MSE(right)}
$$
This ensures splits are chosen based on their ability to reduce error. 
The output is the `best_feature` (column index of the splitting feature) and the `best_threshold` (value of the feature where the split happens).

The same can be done with the method of variance reduction we want to maximize the variance reduction.

$$
  \text{Variance Reduction} = \text{Variance(parent)} - \big[\frac{n_{\text{left}}}{n_{\text{total}}} \cdot \text{Variance(left)} + \frac{n_{\text{right}}}{n_{\text{total}}} \cdot \text{Variance(right)}\big]
$$

It evaluates how well the splits separate the data into subsets that are better at differantiating the target value and making the leaf nodes more pure.

In [317]:
def build_tree(X, y, max_depth, min_samples_split, split_measure, max_features=None, depth=0):

    if depth >= max_depth or len(y) < min_samples_split or mse(y) == 0:
        return np.mean(y)  

    n_features = X.shape[1]
    if max_features == 'sqrt':
        feature_subset = np.random.choice(n_features, size=int(np.sqrt(n_features)), replace=False)
    elif max_features == 'log2':    
        feature_subset = np.random.choice(n_features, size=int(np.log2(n_features)), replace=False)
    else:
        feature_subset = np.random.choice(n_features, size=int(n_features), replace=False)
        
    feature_idx, threshold = find_best_split(X, y, split_measure, feature_subset)
    
    if feature_idx is None:
        return np.mean(y)  
    
    X_left, X_right, y_left, y_right = split_dataset(X, y, feature_idx, threshold)
    left_subtree = build_tree(X_left, y_left, max_depth, min_samples_split, split_measure, depth + 1)
    right_subtree = build_tree(X_right, y_right, max_depth, min_samples_split, split_measure,depth + 1)
    
    return {
        "feature_idx": feature_idx,
        "threshold": threshold,
        "left": left_subtree,
        "right": right_subtree,
    }


- __Stopping Conditions__: Stops if the max depth is reached, if there are fewer samples than min_samples_split, or if the MSE is 0 (all values are the same).
- __Recursive Splitting__: For each split, the function creates a left and right subtree until the stopping conditions are met.
- __Leaf Node__: If the recursion stops, the tree stores the mean value of y for prediction.
- __Feature Selection__: Chooses $\sqrt{n}$ features randomly for each tree, where $n$ is the number of original features

In [318]:
def predict_tree(tree, X):

    if isinstance(tree, dict):

        feature_idx = tree["feature_idx"]
        threshold = tree["threshold"]

        if X[feature_idx] <= threshold:
            return predict_tree(tree["left"], X)
        
        else:
            return predict_tree(tree["right"], X)
        
    else:
        return tree  # Leaf node


Traverses the tree based on the input features until you reach a leaf node.
Returns the mean value of the target variable y at the leaf node.

## Random Forest Regression

In [319]:
class RandomForestRegressor_18:
    
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=10, max_features=None, split_measure='mse'):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.split_measure = split_measure
        self.max_features = max_features
        self.trees = []
        
    def fit(self, X, y):
        bootstraps = make_bootstraps(X, y, n_bootstraps=self.n_trees)
        self.trees = [
            build_tree(X_boot, y_boot, self.max_depth, self.min_samples_split, self.max_features, self.split_measure)
            for X_boot, y_boot in bootstraps
        ]
    
    def predict(self, X):
        predictions = np.array([predict_tree(tree, x) for tree in self.trees for x in X])
        predictions = predictions.reshape(len(self.trees), len(X))

        return np.mean(predictions, axis=0)

## Evaluation

We use for the comparison of the methods the same holdout split with the training set containg 75% of the data.

In [320]:
#X = df_pref_one_hot.drop("Preference", axis=1).values
#y = df_pref_one_hot["Preference"].values

X = df_airfol.drop("y", axis=1).values
y = df_airfol["y"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [329]:
X_abalone = df_abalone.drop("Rings", axis=1).values
y_abalone = df_abalone["Rings"].values

X_train_abalone, X_test_abalone, y_train_abalone, y_test_abalone = train_test_split(X_abalone, y_abalone, test_size=0.25, random_state=42)

For evaluation we use the following three methods:

__Mean Squared Error (MSE)__

The average of the squared differences between the predicted values and the actual values. It gives more weight to larger errors.

- A smaller MSE value indicates that the model’s predictions are close to the actual values.
- Since it's based on squared differences, large prediction errors (outliers) have a greater impact.
- MSE is in the square of the unit of your target variable
- Useful when large errors are particularly undesirable and need to be penalized more heavily.



__Mean Absolute Error (MAE)__

The average of the absolute differences between predicted values and actual values. Unlike MSE, it treats all errors equally, regardless of size.

- A smaller MAE value indicates better model performance.
- MAE is in the same unit as the target variable, making it more interpretable compared to MSE.
- Good for understanding the typical size of prediction errors.
- Less sensitive to outliers compared to MSE.

__R-squared__

The proportion of variance in the target variable that the model explains. It ranges from:
- __1__: Perfect fit (model explains all variance in the data).
- __0__: Model does no better than predicting the mean of the target.
- __Negative__: Model performs worse than simply predicting the mean.


- A higher R-squared (close to 1) indicates a good fit.
- A low or negative R-squared suggests that your model is not capturing the relationship between the features and target effectively.
- Helps understand how well the model explains the variability in the target variable.
- Not ideal for measuring absolute error but useful for comparing models.

### Our Regressor

In [330]:
rf_18 = RandomForestRegressor_18(n_trees=20, max_depth=5, min_samples_split=3, split_measure='variance')
rf_18.fit(X_train, y_train)

predictions_18 = rf_18.predict(X_test)

Time run for mountains vs. beaches dataset: 34m 49s

Time run for Airfoil dataset:

In [331]:
mse_18 = mean_squared_error(y_test, predictions_18)
mae_18 = mean_absolute_error(y_test, predictions_18)
r2_18 = r2_score(y_test, predictions_18)

print("Model Performance:")
print(f"Mean Squared Error: {mse_18:.2f}")
print(f"Mean Absolute Error: {mae_18:.2f}")
print(f"R-squared: {r2_18:.2f}")

Model Performance:
Mean Squared Error: 47.78
Mean Absolute Error: 5.63
R-squared: -0.00


### Fitting abalone dataset

In [332]:
from time import process_time, time

#rf_18_abalone = RandomForestRegressor_18(n_trees=20, max_depth=5, max_features='sqrt', min_samples_split=3, split_measure='variance')
rf_18_abalone = RandomForestRegressor_18(n_trees=20, max_depth=5, min_samples_split=3, split_measure='variance')
rf_18_abalone.fit(X_train_abalone, y_train_abalone)
predictions_18_abalone = rf_18_abalone.predict(X_test_abalone)

In [328]:
mse_18 = mean_squared_error(y_test_abalone, predictions_18_abalone)
mae_18 = mean_absolute_error(y_test_abalone, predictions_18_abalone)
r2_18 = r2_score(y_test_abalone, predictions_18_abalone)

print("Model Performance:")
print(f"Mean Squared Error: {mse_18:.2f}")
print(f"Mean Absolute Error: {mae_18:.2f}")
print(f"R-squared: {r2_18:.2f}")

Model Performance:
Mean Squared Error: 11.03
Mean Absolute Error: 2.43
R-squared: -0.00


Metrics for abalone dataset:

Model Performance:
Mean Squared Error: 
Mean Absolute Error: 
R-squared: 

### SkLearn Regressor

In [42]:
from sklearn.ensemble import RandomForestRegressor

In [43]:
rf_sklearn = RandomForestRegressor(n_estimators=20, max_depth = 5, min_samples_split = 3)
rf_sklearn.fit(X_train, y_train)

predictions_sklearn = rf_sklearn.predict(X_test)

In [44]:
mse_sklearn = mean_squared_error(y_test, predictions_sklearn)
mae_sklearn = mean_absolute_error(y_test, predictions_sklearn)
r2_sklearn = r2_score(y_test, predictions_sklearn)

print("Model Performance:")
print(f"Mean Squared Error: {mse_sklearn:.2f}")
print(f"Mean Absolute Error: {mae_sklearn:.2f}")
print(f"R-squared: {r2_sklearn:.2f}")

Model Performance:
Mean Squared Error: 0.01
Mean Absolute Error: 0.04
R-squared: 0.94


Metrics for mountains vs. beaches dataset:

Model Performance:
Mean Squared Error: 0.01
Mean Absolute Error: 0.04
R-squared: 0.94

Metrics run for Airfoil dataset:

### ChatGPT Regressor

In [45]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        data = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        self.tree = self._build_tree(data)

    def predict(self, X):
        return np.array([self._predict_row(row, self.tree) for row in X])

    def _build_tree(self, data, depth=0):
        X, y = data[:, :-1], data[:, -1]
        num_samples, num_features = X.shape

        # Stopping conditions
        if num_samples < self.min_samples_split or (self.max_depth and depth >= self.max_depth):
            return np.mean(y)

        feature_idx, threshold = self._find_best_split(X, y)
        if feature_idx is None:
            return np.mean(y)

        left_idxs, right_idxs = self._split(X[:, feature_idx], threshold)
        left = self._build_tree(data[left_idxs], depth + 1)
        right = self._build_tree(data[right_idxs], depth + 1)
        return {"feature_idx": feature_idx, "threshold": threshold, "left": left, "right": right}

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape
        best_mse = float("inf")
        split_idx, split_thresh = None, None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                mse = self._mean_squared_error_split(X[:, feature_idx], y, threshold)
                if mse < best_mse:
                    best_mse, split_idx, split_thresh = mse, feature_idx, threshold

        return split_idx, split_thresh

    def _mean_squared_error_split(self, feature, y, threshold):
        left_idxs, right_idxs = self._split(feature, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return float("inf")

        y_left, y_right = y[left_idxs], y[right_idxs]
        mse_left = np.mean((y_left - np.mean(y_left))**2)
        mse_right = np.mean((y_right - np.mean(y_right))**2)

        n = len(y)
        n_left, n_right = len(y_left), len(y_right)
        mse = (n_left / n) * mse_left + (n_right / n) * mse_right

        return mse

    def _split(self, feature, threshold):
        left_idxs = np.where(feature <= threshold)[0]
        right_idxs = np.where(feature > threshold)[0]
        return left_idxs, right_idxs

    def _predict_row(self, row, tree):
        if isinstance(tree, dict):
            if row[tree["feature_idx"]] <= tree["threshold"]:
                return self._predict_row(row, tree["left"])
            else:
                return self._predict_row(row, tree["right"])
        return tree


class RandomForestRegressor:
    def __init__(self, n_trees=10, max_depth=None, min_samples_split=2, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples, n_features = X.shape
        for _ in range(self.n_trees):
            bootstrap_idxs = np.random.choice(n_samples, n_samples, replace=True)
            bootstrap_X = X[bootstrap_idxs]
            bootstrap_y = y[bootstrap_idxs]

            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            if self.max_features is not None:
                feature_idxs = np.random.choice(n_features, self.max_features, replace=False)
                tree.fit(bootstrap_X[:, feature_idxs], bootstrap_y)
                tree.feature_idxs = feature_idxs
            else:
                tree.fit(bootstrap_X, bootstrap_y)
                tree.feature_idxs = None
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X[:, tree.feature_idxs] if tree.feature_idxs is not None else X)
                                      for tree in self.trees])
        return np.mean(tree_predictions, axis=0)

In [46]:
rf_chatgpt = RandomForestRegressor(n_trees=20, max_depth=5, min_samples_split=3)
rf_chatgpt.fit(X_train, y_train)
predictions_chatgpt = rf_chatgpt.predict(X_test)

KeyboardInterrupt: 

Time run for mountains vs. beaches dataset:

Time run for Airfoil dataset:

In [98]:
mse_chatgpt = mean_squared_error(y_test, predictions_chatgpt)
mae_chatgpt = mean_absolute_error(y_test, predictions_chatgpt)
r2_chatgpt = r2_score(y_test, predictions_chatgpt)

print("Model Performance:")
print(f"Mean Squared Error: {mse_chatgpt:.2f}")
print(f"Mean Absolute Error: {mae_chatgpt:.2f}")
print(f"R-squared: {r2_chatgpt:.2f}")

Model Performance:
Mean Squared Error: 2895.24
Mean Absolute Error: 44.53
R-squared: 0.45


Metrics for mountains vs. beaches dataset:

Metrics run for Airfoil dataset:

## Notes

Max Depth: Prevents trees from growing too deep, which could lead to overfitting.

Min Samples Split: Controls the smallest group size allowed for further splitting, preventing unnecessary splits.


**Key Insights from These Metrics**
1. **MSE**:
   - If it's high, your model is making some large errors that need to be addressed.
   - If it's low, your model is capturing most of the relationship.

2. **MAE**:
   - Directly tells you the average prediction error. 
   - Compare it to the scale of your target variable; if MAE is relatively low, the model is performing well.

3. **R²**:
   - A high R² suggests the model explains a significant portion of the target variable's variance.
   - If R² is low (or negative), consider if the features are truly predictive of the target or if the model is too simple/complex.


**Next Steps Based on Metrics**

- **High MSE or MAE**:
  - Investigate outliers, or whether the model needs better hyperparameter tuning.
  - Consider adding more predictive features or improving feature engineering.

- **Low R²**:
  - Evaluate if features are relevant or add more features to capture the variance.
  - Consider if the model is underfitting or overfitting:
    - Underfitting: Increase `max_depth`, add more `n_trees`.
    - Overfitting: Decrease `max_depth` or regularize.
